# Particule Swam Optimization implementation

PSO 

In [194]:
from functions import *
import numpy as np
import matplotlib as plt
import time

In [200]:
class F1_Shifted_Sphere:
    
    nbFitness = 0
    
    def __init__(self, dim, bounds):
        self.dim = dim
        self.bounds=bounds
        F1_Shifted_Sphere.nbFitness = 0

    def fitness(self, x):
        z=0
        F = 0
        for i in range(0,x.shape[0]):
            z = x[i] - sphere[i]
            F = F + z*z;
        F1_Shifted_Sphere.nbFitness += 1
        
        return F + f_bias[0]

# definition of a class for Particle
class Particle:
    def __init__(self, problem):
        self.problem=problem
        self.position = np.random.uniform(low=self.problem.bounds[0], high=self.problem.bounds[1], size=self.problem.dim)
        self.velocity = np.random.uniform(low=-(self.problem.bounds[1]-self.problem.bounds[0]) \
                                        , high=(self.problem.bounds[1]-self.problem.bounds[0]), size=self.problem.dim)
        self.best_part_pos = self.position.copy()
        self.fitness = self.problem.fitness(self.position)
        self.best_fitness = self.fitness.copy()

    def moveParticule (self, swarm):
        #print (self.position)
        self.velocity =  swarm.w * self.velocity  + \
            swarm.c1 * np.random.uniform(low=0, high=(self.best_part_pos - self.position), size=self.problem.dim) + \
            swarm.c2 * np.random.uniform(low=0, high=(swarm.best_swarm_position - self.position), size=self.problem.dim)
        self.position = self.position + self.velocity
        self.position=np.minimum(self.position, self.problem.bounds[1])
        self.position=np.maximum(self.position, self.problem.bounds[0])
        #print (self.position)
        self.fitness = self.problem.fitness(self.position)
        #print(self.fitness)
        if self.fitness < self.best_fitness:
            self.best_fitness = self.fitness
            self.best_part_pos = self.position
            
            
# definition of a class for Swarm
class Swarm:

    def __init__(self, problem, numOfParticule=5, w=0.7, c1=1.4, c2=1.4):
        self.w=w
        self.c1=c1
        self.c2=c2
        self.problem=problem
        self.swarm_list = [Particle(problem) for i in range(numOfParticule)]
        self.best_swarm_position = self.swarm_list[0].position
        self.best_swarm_fitness = self.swarm_list[0].fitness
        for particule in self.swarm_list:
            if particule.fitness < self.best_swarm_fitness:
                self.best_swarm_position = particule.position
                self.best_swarm_fitness = particule.fitness
        

        
    def moveSwarm(self):
        for particule in self.swarm_list:
            particule.moveParticule(self) # update the position of particle
            if particule.fitness < self.best_swarm_fitness:  # check the position if it's best for swarm
                self.best_swarm_position = particule.position
                self.best_swarm_fitness = particule.fitness

# definition of a class for Optimize
class Optimize:
        def __init__(self,swarm, numOfEpochs=0, epsilon=0, N=100 ):
            self.epsilon=epsilon
            self.N=N
            self.swarm=swarm
            
        
        def run(self):
            ts=time.time()
            progress = []
            previousMin=self.swarm.best_swarm_fitness
            progress.append(previousMin)
            carryOn = True
            i = 0
            while carryOn: 
                self.swarm.moveSwarm()
                progress.append(self.swarm.best_swarm_fitness)
                # print('Run: {0} | Best fitness: {1}'.format(i, swarm.best_swarm_fitness))
                if previousMin > self.swarm.best_swarm_fitness:
                    previousMin=self.swarm.best_swarm_fitness
                if i>= self.N :
                    if (progress[- self.N] - progress[-1]) <= self.epsilon:
                        carryOn = False
                i += 1
            return progress, self.swarm.problem.nbFitness, time.time()-ts
        

In [201]:

problem=F1_Shifted_Sphere(50, [-100,100])
swarm=Swarm(problem, numOfParticule=30, w=0.2, c1=1.5, c2=1.2)

optimize = Optimize(swarm, epsilon=0, N=30)
res = optimize.run()
res[1]

6600

In [137]:
problem=F1_Shifted_Sphere(500, [-100,100])
swarm=Swarm(problem, numOfParticule=1000, w=0.2, c1=1.5, c2=1.2)
optimize = Optimize(swarm, epsilon=0, N=300)
res = optimize.run()
res = np.array(res)
res[-1]

320118.42162031354